In [1]:
import pandas as pd
import datetime
import csv

from DATA225utils import make_connection, dataframe_query 

In [2]:
# connecttomysql.ini configuration file connects the python to mysql 
# so that we can create the database through python as well.
conn = make_connection(config_file = 'configFiles/local_snps_db.ini')
cursor = conn.cursor()

In [3]:
sql ="""
        SELECT DISTINCT(PGS.player_id) FROM player_game_stats PGS
        WHERE PGS.player_id NOT IN (SELECT player_id FROM players) """

df = dataframe_query(conn, sql)[1]

In [4]:
df.columns

Index(['player_id'], dtype='object')

In [5]:
from nba_api.stats.endpoints import CommonPlayerInfo

playerid = '696'
def getMissingPlayerInfo(player_id):
    return CommonPlayerInfo(player_id).get_data_frames()[0][['FIRST_NAME', 'LAST_NAME', 'TEAM_ID', 'HEIGHT', 'DRAFT_YEAR']]

In [6]:
import numpy as np

df["FIRST_NAME"] = np.nan
df["LAST_NAME"] = np.nan
df["TEAM_ID"] = np.nan
df["HEIGHT"] = np.nan
df["DRAFT_YEAR"] = np.nan

In [7]:
df

,player_id,FIRST_NAME,LAST_NAME,TEAM_ID,HEIGHT,DRAFT_YEAR
0,696,NaN,NaN,NaN,NaN,NaN
1,731,NaN,NaN,NaN,NaN,NaN
2,762,NaN,NaN,NaN,NaN,NaN
3,1052,NaN,NaN,NaN,NaN,NaN
4,1504,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
913,1628238,NaN,NaN,NaN,NaN,NaN
914,1629873,NaN,NaN,NaN,NaN,NaN
915,1630994,NaN,NaN,NaN,NaN,NaN
916,1628221,NaN,NaN,NaN,NaN,NaN


In [8]:
import time

for i, player in df.iterrows():
    try:
        playerInfo = getMissingPlayerInfo(int(float(player.player_id)))        
        df.loc[i, "FIRST_NAME"] = playerInfo["FIRST_NAME"][0]
        df.loc[i, "LAST_NAME"] = playerInfo["LAST_NAME"][0]
        df.loc[i, "TEAM_ID"] = playerInfo["TEAM_ID"][0]
        df.loc[i, "HEIGHT"] = playerInfo["HEIGHT"][0]
        df.loc[i, "DRAFT_YEAR"] = playerInfo["DRAFT_YEAR"][0]
        time.sleep(0.5)
    except:
        continue
    

In [9]:
df

,player_id,FIRST_NAME,LAST_NAME,TEAM_ID,HEIGHT,DRAFT_YEAR
0,696,Travis,Best,1.610613e+09,5-11,1995
1,731,Greg,Ostertag,0.000000e+00,7-2,1995
2,762,Shawn,Bradley,1.610613e+09,7-6,1993
3,1052,Ben,Handlogten,1.610613e+09,6-10,Undrafted
4,1504,Danny,Fortson,1.610613e+09,6-8,1997
...,...,...,...,...,...,...
913,1628238,Paris,Bass,0.000000e+00,6-8,Undrafted
914,1629873,Jaysean,Paige,0.000000e+00,6-2,Undrafted
915,1630994,Gabriel,Lundberg,1.610613e+09,6-4,Undrafted
916,1628221,Gabe,York,1.610613e+09,6-3,Undrafted


In [10]:
df.isna().sum()

player_id      0
FIRST_NAME    64
LAST_NAME     64
TEAM_ID       64
HEIGHT        64
DRAFT_YEAR    64
dtype: int64

In [11]:
df.shape

(918, 6)

In [12]:
df.loc[:, "FULL_NAME"] = df.FIRST_NAME+" "+df.LAST_NAME

In [13]:
df.loc[:, "SEASON"] = "2014"

In [14]:
df.dropna(inplace = True)

In [15]:
for i, player in df.iterrows():
    if player.HEIGHT != '':
#     try:
        df.loc[i, "HEIGHT"] = (float(player.HEIGHT.split("-")[0]+"."+player.HEIGHT.split("-")[1]))*30.48
    else:
        df.loc[i, "HEIGHT"] = np.nan

In [16]:
df.isna().sum()

player_id       0
FIRST_NAME      0
LAST_NAME       0
TEAM_ID         0
HEIGHT        165
DRAFT_YEAR      0
FULL_NAME       0
SEASON          0
dtype: int64

In [17]:
df

,player_id,FIRST_NAME,LAST_NAME,TEAM_ID,HEIGHT,DRAFT_YEAR,FULL_NAME,SEASON
0,696,Travis,Best,1.610613e+09,155.7528,1995,Travis Best,2014
1,731,Greg,Ostertag,0.000000e+00,219.456,1995,Greg Ostertag,2014
2,762,Shawn,Bradley,1.610613e+09,231.648,1993,Shawn Bradley,2014
3,1052,Ben,Handlogten,1.610613e+09,185.928,Undrafted,Ben Handlogten,2014
4,1504,Danny,Fortson,1.610613e+09,207.264,1997,Danny Fortson,2014
...,...,...,...,...,...,...,...,...
913,1628238,Paris,Bass,0.000000e+00,207.264,Undrafted,Paris Bass,2014
914,1629873,Jaysean,Paige,0.000000e+00,188.976,Undrafted,Jaysean Paige,2014
915,1630994,Gabriel,Lundberg,1.610613e+09,195.072,Undrafted,Gabriel Lundberg,2014
916,1628221,Gabe,York,1.610613e+09,192.024,Undrafted,Gabe York,2014


In [19]:
df.to_csv("data/missingPlayerInfo.csv")

In [18]:
sql_insert = (   """
            INSERT INTO players
            VALUES (%s, %s, %s, %s, %s, %s)
            """
        )

for index, row in df.iterrows():
    values = (row['FULL_NAME'], row['TEAM_ID'], row['player_id'], \
              row['SEASON'], row['HEIGHT'], row['DRAFT_YEAR'])
    cursor.execute(sql_insert, values)
    
conn.commit()

In [1]:
cursor.close()
conn.close()

NameError: name 'cursor' is not defined